<a href="https://colab.research.google.com/github/Monika14326/FFML-labs/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits can indeed give more consistent and reliable results. This is commonly referred to as cross-validation or k-fold cross-validation. Here's why:

1. Reduces Variance
When you train and evaluate your model on a single validation set, the performance you measure is dependent on that specific split of the data. If the validation set is particularly easy or hard, the accuracy may be skewed. By averaging over multiple splits, you reduce the influence of any individual data split, which leads to a more stable and robust estimate of the model's performance.
2. Better Generalization Estimate
By testing the model on multiple different sets of data, you get a more representative sense of how the model will perform across the entire dataset. If the data splits are well-randomized, averaging the performance across splits helps smooth out fluctuations that might occur from idiosyncrasies in any specific split.
3. Prevents Overfitting
If you evaluate on a single validation split, there’s a risk that the model could overfit to the specific characteristics of that split. Using multiple splits ensures the model is evaluated on a broader variety of data, making the results less susceptible to overfitting.
4. Increased Reliability
A single validation set might give a high or low accuracy purely due to random chance in the data partitioning. Averaging across multiple splits helps to ensure that the performance measure reflects the model's true capability rather than being influenced by a single, potentially unrepresentative, split.
5. Helps Identify Model Instability
Averaging over multiple splits also helps identify if the model's performance is highly sensitive to the choice of validation data. If there is significant variation in the results across splits, it could indicate that the model is unstable and not generalizing well.
How it's Done:
In k-fold cross-validation, the dataset is split into k subsets. The model is trained k times, each time using a different subset as the validation set and the remaining k-1 subsets as the training set. The performance (e.g., accuracy) is averaged over the k runs to give a more robust evaluation.
2. Does it give more accurate estimate of test accuracy?
Averaging the validation accuracy across multiple splits does not directly provide a more accurate estimate of test accuracy, but it does improve the estimate of model performance on unseen data in several ways. Here's a more nuanced explanation:

Key Points:
Difference Between Validation and Test Accuracy:

Validation accuracy measures the model's performance on data that was not used during training, but is still part of the data available during model selection and hyperparameter tuning.
Test accuracy measures the model's performance on completely unseen data that was not part of the training or validation sets. This is the true measure of how well the model generalizes to new data.
Role of Cross-Validation:

Averaging across multiple splits (e.g., in k-fold cross-validation) gives a more reliable estimate of the validation accuracy. This is because it reduces the variance associated with a single validation set, ensuring that your model’s performance is not overly sensitive to the specific choice of validation data.
Cross-validation improves the reliability of validation accuracy as a proxy for model performance, but it still reflects the model’s ability to generalize within the given dataset. Cross-validation does not provide a direct estimate of test accuracy.
Better Estimation of Generalization Performance:

The averaged validation accuracy from cross-validation is often used to choose the best model, hyperparameters, or features. This leads to a better model, which might, in turn, result in a more accurate test set performance.
However, it's important to note that the test set (ideally) should remain untouched during model selection. If you're using your test set to make decisions about your model (for example, if you perform multiple rounds of hyperparameter tuning based on test performance), this can lead to data leakage and an overly optimistic estimate of test accuracy.
Final Test Accuracy Estimate:

After cross-validation and model selection, the final estimate of test accuracy should be done using a separate, holdout test set that was never involved in training or validation. This test set provides the true measure of generalization.
The cross-validation accuracy is a good approximation of test accuracy, but it's not identical. Cross-validation essentially tries to mimic the process of training and testing on unseen data, but it still uses data within the same dataset.
Why Cross-Validation Improves Estimates:
Reduces Bias in Model Selection: By averaging over multiple splits, you prevent the risk of selecting a model that performs well on a single, possibly unrepresentative validation set. This reduces the chance of overfitting or selecting a model that might only perform well on a specific subset of the data.

Helps with Hyperparameter Tuning: When you're tuning hyperparameters, averaging across multiple validation splits helps to ensure that the chosen set of parameters is more robust and not overly fitted to a specific subset of the data.


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Yes, increasing the number of iterations in techniques like cross-validation (specifically, increasing the number of folds in k-fold cross-validation) generally leads to a better estimate of the model's performance. However, the impact depends on the context and the balance between computational cost and the accuracy of the estimate. Here's a detailed breakdown of how the number of iterations (or folds) influences the estimate:

1. Effect of Increasing the Number of Iterations (or Folds)
More Accurate Estimate of Performance:

As the number of iterations or folds increases, the estimate of the model's performance becomes more reliable and stable. This is because:
Each fold provides an additional independent estimate of how the model performs on unseen data.
More folds mean more diverse validation sets, which reduces the variability of the performance estimate.
For example, 10-fold cross-validation typically provides a good balance between accuracy and computational cost. However, increasing this to 20 or 30 folds could give an even more precise estimate, especially for smaller datasets.
Reduces the Variance in the Estimate:

With more iterations (folds), you reduce the potential bias that might arise from a single validation set. This is particularly important when the dataset is small or the model’s performance varies significantly across different subsets of the data.
Fewer folds (e.g., 3-fold or 5-fold) might result in a performance estimate that’s more sensitive to the specifics of the chosen splits, while more folds (e.g., 10 or 20) average out the performance across more splits, leading to a less variable estimate.
Better Coverage of the Dataset:

With more folds, each data point gets used for both training and validation multiple times, ensuring that the estimate of model performance is less likely to be biased by the particular characteristics of the data in any one split.
2. Diminishing Returns with More Folds
While increasing the number of iterations or folds generally improves the estimate, there are diminishing returns. After a certain point, the benefits of adding more folds decrease:
With very high numbers of folds, such as 50 or 100, the variance reduction becomes smaller, and the performance estimate will be quite stable.
However, this comes at the cost of increased computational time. More folds mean that the model must be trained more times, which increases the overall computational cost. The trade-off between computational cost and precision of the estimate must be considered based on available resources.
3. Bias-Variance Tradeoff
Bias: The estimate is less biased with more folds because it’s based on a larger number of independent training-validation splits.

Variance: With fewer folds, the variance of the estimate can be higher because you’re relying on fewer splits. More folds reduce this variance.

The optimal number of folds depends on the size of the dataset:

For larger datasets, even fewer folds (e.g., 5 or 10) are usually sufficient to get a good estimate.
For smaller datasets, increasing the number of folds (e.g., using 20 or more) can improve the reliability of the performance estimate.
4. Computational Efficiency Considerations
Increasing folds increases the computational burden since the model is trained and evaluated multiple times. So, the more iterations (or folds), the longer the process.
In practice, cross-validation is a balance: 10-fold cross-validation is a commonly used standard because it offers a good trade-off between a reliable estimate and reasonable computational cost.
5. Leave-One-Out Cross-Validation (LOOCV)
In the extreme case, Leave-One-Out Cross-Validation (LOOCV) is used, where each data point is used as a single validation set and the model is trained on the remaining data. While this approach can provide an almost unbiased estimate of model performance (as it uses nearly all of the data for training each time), it’s computationally expensive for larger datasets and doesn’t always improve the estimate significantly beyond 10-fold cross-validation for many models.

4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations (or folds) in cross-validation does **not** directly solve the issue of having a very small training or validation dataset. However, it can help mitigate some of the challenges posed by small datasets, though there are some important nuances to consider.

Here’s a breakdown of how increasing the number of iterations or folds in cross-validation interacts with small datasets and what you can do to handle such situations effectively:

### 1. **Small Training Dataset:**
   - **Problem**: If the training dataset is very small, the model might not have enough data to learn meaningful patterns, leading to poor generalization.
   - **Effect of Increasing Folds**: Increasing the number of folds (for example, moving from 5-fold to 10-fold cross-validation) allows each data point to be used as both training and validation data multiple times. This means:
     - The model gets to train on nearly all of the data in each iteration, which maximizes the use of the limited data.
     - Each data point is used in the training set more frequently, but the model still gets tested on different subsets during validation. This can give a more reliable estimate of performance.

     However, **more folds don’t fundamentally change the problem** of limited data. You are still training on a small amount of data in each fold, which could lead to **underfitting** (where the model doesn't capture important patterns) or **high variance** in performance due to the small size of each training set.

### 2. **Small Validation Dataset:**
   - **Problem**: If the validation dataset is too small, the performance estimates might be too noisy and unreliable, reflecting the variability of the small subset rather than the model's true generalization ability.
   - **Effect of Increasing Folds**: Increasing the number of folds reduces the size of each individual validation set, but the number of times a point is validated on increases. In cross-validation:
     - Each data point is used as a validation point in some fold, which allows you to get a more comprehensive view of model performance.
     - While it helps to reduce the variance of the validation performance estimate (by averaging over multiple folds), it doesn't solve the problem of the validation set being small in a single fold. Essentially, each individual validation set is smaller, but you are testing more combinations across folds.

     If the dataset is very small, this approach still has limitations, as the model’s performance can be highly dependent on which data points end up in the validation set.

### 3. **Impact of Small Datasets on Cross-Validation:**
   - **Variance in Results**: With very small datasets, cross-validation can still give high variance in performance estimates. If your dataset has only a few hundred or fewer data points, each fold may not be representative of the full data distribution, and performance metrics may fluctuate.
   - **Bias in Results**: Small datasets can also lead to **bias** in the performance estimate. If the dataset is highly imbalanced or non-representative, cross-validation may not help because the splits might still be unbalanced or not reflect the diversity of the full data.

### 4. **Alternatives and Complementary Approaches:**
   - **Data Augmentation**: For small training datasets, techniques like **data augmentation** (in tasks like image classification) or synthetic data generation can help artificially increase the size of the dataset. This can provide more meaningful data for the model to learn from.
   - **Transfer Learning**: In cases of small datasets, you can use **pre-trained models** on larger datasets and fine-tune them on your small dataset. This helps the model leverage prior knowledge learned from a larger dataset.
   - **Leave-One-Out Cross-Validation (LOOCV)**: If the dataset is very small, **LOOCV** might be more effective than k-fold cross-validation. In LOOCV, each data point is used for validation exactly once, and the model is trained on all the other points. This maximizes the amount of training data available for each model run, though it can be computationally expensive.
   - **Regularization**: To prevent overfitting on small datasets, applying **regularization** (e.g., L1, L2 regularization) can help the model generalize better despite the small data size.

### 5. **Handling Imbalanced Datasets:**
   - If your small dataset is also **imbalanced** (for example, in classification tasks where one class is underrepresented), you might consider techniques like **stratified k-fold cross-validation**. This ensures that each fold has a balanced distribution of classes, reducing the risk of bias in your performance estimates.

### Summary:
- Increasing the number of iterations or folds in cross-validation **does not fundamentally solve the issue of having very small datasets**, whether for training or validation.
- More folds do help make the performance estimate more reliable, especially for small datasets, but the overall problem of limited data remains, and the model might still face challenges like **underfitting** or **overfitting**.
- To effectively deal with small datasets, consider **data augmentation**, **transfer learning**, **regularization**, and perhaps **LOOCV**. Cross-validation can still be valuable in these cases, but other techniques should be used in conjunction to improve the model's performance and generalization.




> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.